In [1]:
from sklearn import datasets


In [2]:
digits = datasets.load_digits()

In [3]:
X = digits.data
y = digits.target

X.shape

(1797, 64)

In [4]:
y.shape

(1797,)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [6]:
from kNN import KNNClassifier
knn = KNNClassifier(3)
knn.fit(X_train, y_train)
y_predict = knn.predict(X_test)

In [7]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_predict, y_test)
score

0.9888888888888889

In [8]:
sum(y_predict == y_test) / len(y_test)

0.9888888888888889

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn_sk = KNeighborsClassifier(n_neighbors=3)
knn_sk.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [10]:
y_predict = knn_sk.predict(X_test)
accuracy_score(y_predict, y_test)

0.9888888888888889